# Predicting Stock Price Using Historical Data

In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import seaborn as sns

# Function to fetch stock data
def fetch_stock_data():
    stock_symbol = symbol_entry.get()
    start_date = start_date_entry.get()
    end_date = end_date_entry.get()

    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

    return stock_data

# Function to prepare data for LSTM
def prepare_data(stock_data):
    try:
        # Feature scaling
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))

        # Create sequences for LSTM
        sequence_length = 10  # Number of past days used for prediction
        sequences = []
        for i in range(len(scaled_data) - sequence_length):
            sequence = scaled_data[i : i + sequence_length + 1]
            sequences.append(sequence)

        sequences = np.array(sequences)

        # Split into input (X) and output (y) variables
        X = sequences[:, :-1]
        y = sequences[:, -1]

        return X, y, scaler
    except Exception as e:
        # If an exception occurs (e.g., invalid symbol), display an error message
        prediction_label.config(text="")
        error_label.config(text="Error: Invalid Stock Symbol or Date Range")

# Function to build and train the LSTM model
def build_lstm_model(X_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_shape[1], X_shape[2])))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Function to update the graph
def update_graph(stock_data, next_day_prediction):
    plt.clf()  # Clear the previous plot

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(stock_data.index, stock_data['Close'], label="Actual Prices")
    ax.axhline(y=next_day_prediction, color='r', linestyle='--', label="Next Day Prediction")
    ax.set_xlabel('Date')
    ax.set_ylabel('Stock Price')
    ax.set_title('Stock Price Chart')
    ax.legend()
    plt.tight_layout

    # Convert the plot to a Tkinter Canvas widget
    canvas = FigureCanvasTkAgg(fig, master=graph_frame)
    canvas.draw()

    if hasattr(update_graph, 'plot_widget'):
        # If the widget already exists, delete it before updating
        update_graph.plot_widget.get_tk_widget().destroy()

    update_graph.plot_widget = canvas
    update_graph.plot_widget.get_tk_widget().pack(fill=tk.BOTH, expand=True)


# Function to update the table
def update_table(stock_data):
    for row in tree.get_children():
        tree.delete(row)

    for index, row in stock_data.iterrows():
        tree.insert('', 'end', values=(index, row['Open'], row['High'], row['Low'], row['Close'], row['Volume']))


# Function to make predictions using the LSTM model
def predict_stock():
    try:
        stock_data = fetch_stock_data()
        X, y, scaler = prepare_data(stock_data)

        # Reshape X for LSTM input (batch size, timesteps, features)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        # Build and train LSTM model (you need to implement this based on your use case)
        model = build_lstm_model(X.shape)
        model.fit(X, y, epochs=100, batch_size=32, verbose=0)

        # Make predictions for the next day
        last_sequence = X[-1]
        last_sequence = last_sequence.reshape((1, last_sequence.shape[0], 1))
        next_day_prediction = model.predict(last_sequence)
        next_day_prediction = scaler.inverse_transform(next_day_prediction)[0][0]

        # Display prediction
        prediction_label.config(text=f"Next Day Prediction: {next_day_prediction:.2f}")

        # Update graph with actual and predicted stock prices
        update_graph(stock_data, next_day_prediction)

        # Update table with stock data
        update_table(stock_data)

        # Clear any previous error message
        error_label.config(text="")

    except Exception as e:
        # If an exception occurs (e.g., invalid symbol), display an error message
        prediction_label.config(text="")
        error_label.config(text="Error: Invalid Stock Symbol or Date Range")
# GUI Setup
root = tk.Tk()
root.title('Stock Predictor App')

# Input Frame
input_frame = ttk.Frame(root)
input_frame.pack(padx=10, pady=10)

symbol_label = ttk.Label(input_frame, text='Stock Symbol:',font=("Helvetica", 10,'bold'))
symbol_label.grid(row=0, column=0, padx=5, pady=5)
symbol_entry = ttk.Entry(input_frame)
symbol_entry.grid(row=0, column=1, padx=5, pady=5)

start_date_label = ttk.Label(input_frame, text='Start Date:(YYYY-MM-DD)',font=("Helvetica", 10,'bold'))
start_date_label.grid(row=1, column=0, padx=5, pady=5)
start_date_entry = ttk.Entry(input_frame)
start_date_entry.grid(row=1, column=1, padx=5, pady=5)

end_date_label = ttk.Label(input_frame, text='End Date:(YYYY-MM-DD)',font=("Helvetica", 10,'bold'))
end_date_label.grid(row=2, column=0, padx=5, pady=5)
end_date_entry = ttk.Entry(input_frame)
end_date_entry.grid(row=2, column=1, padx=5, pady=5)

predict_button = ttk.Button(input_frame, text='Predict', command=predict_stock)
predict_button.grid(row=3, column=0, columnspan=2, padx=5, pady=5)

prediction_label = ttk.Label(input_frame, text="")
prediction_label.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

error_label = ttk.Label(input_frame, text="", foreground="red")
error_label.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

# Table Frame
table_frame = ttk.Frame(root)
table_frame.pack(padx=10, pady=5)

tree = ttk.Treeview(table_frame, columns=('Date', 'Open', 'High', 'Low', 'Close', 'Volume'))
tree.heading('#1', text='Date')
tree.heading('#2', text='Open')
tree.heading('#3', text='High')
tree.heading('#4', text='Low')
tree.heading('#5', text='Close')
tree.heading('#6', text='Volume')
tree.pack()

# Graph Frame
graph_frame = ttk.Frame(root)
graph_frame.pack(padx=10, pady=5)

root.mainloop()

[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 1s 507ms/step
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 491ms/step
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 477ms/step
